**SECTION 1**

In [ ]:
!apt-get update -y

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = (SparkSession
.builder
.appName("<app_name>")
.getOrCreate())

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
chess_info = spark.read.csv("/content/gdrive/MyDrive/Groker/chess_wc_history_game_info.csv",header="true")

In [ ]:
chess_moves = spark.read.csv("/content/gdrive/MyDrive/Groker/chess_wc_history_moves.csv",header="true")

In [ ]:
eco_codes = spark.read.csv("/content/gdrive/MyDrive/Groker/eco_codes.csv",header="true")

In [ ]:
from pyspark.sql.functions import split,col 

**SECTION 2**

1. List of Winners of Each World champions Trophy Hint: Total Result of all rounds of Tournament for that player is considered as that player's
Score/Result.
Result attributes: winner, tournament_name

In [ ]:
chess_info.show(truncate=False)

+------------------------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+------------------------+---------------+
|game_id                             |game_order|event   |site      |date_played|round|white           |black           |result |white_elo|black_elo|white_title|black_title|winner   |winner_elo|loser           |loser_elo|winner_loser_elo_diff|eco|date_created            |tournament_name|
+------------------------------------+----------+--------+----------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+------------------------+---------------+
|86e0b7f5-7b94-4ae3-97c8-317371622795|1         |WCh 2021|Dubai UAE |2021.11.26 |1    |Nepomniachtchi,I|Carlsen,M       |1/2-1/2|2782

In [ ]:
#Splitting winner column to get the first name
chess_info1=chess_info.withColumn("winner_split",split(col("winner"),"[,]"))\
          .withColumn("winner1",col("winner_split")[0]).drop("winner_split","winner")

In [ ]:
chess_info1.show(4)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|winner1|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| Worl

In [ ]:
che =chess_info1.groupBy("tournament_name","winner1").count().where("winner1 !='draw'")

In [ ]:
import pyspark.sql.functions as F

result1 = che.groupBy("tournament_name") \
    .agg(F.max(F.struct("count", "winner1")).alias("max")) \
    .selectExpr("tournament_name","max.winner1 as winner", "max.count as winning_count")
wc=result1

In [ ]:
result_1=result1.select("tournament_name","winner")
result_1.show(5)

+---------------+-------+
|tournament_name| winner|
+---------------+-------+
| WorldChamp2004|   Leko|
| WorldChamp1894| Lasker|
| WorldChamp2013|Carlsen|
|  FideChamp2000| Shirov|
| WorldChamp2007|  Anand|
+---------------+-------+
only showing top 5 rows



2. List of Players with number of times they have won Tournament in descending order(Max to min).
Result attributes: player_name, number_of_wins

In [ ]:
result2=wc.sort('winning_count',ascending=False)
result2=result2.withColumnRenamed("winning_count","number_of_wins")
result2=result2.withColumnRenamed("winner","player_name")
result2=result2.select("player_name","number_of_wins")

In [ ]:
result2.show(15)

+------------+--------------+
| player_name|number_of_wins|
+------------+--------------+
|Kasimdzhanov|            13|
|  Ponomariov|            12|
|   Khalifman|            12|
|    Alekhine|            11|
|    Steinitz|            10|
|    Steinitz|            10|
|    Alekhine|            10|
|       Anand|            10|
|      Lasker|            10|
|      Lasker|            10|
|   Botvinnik|            10|
|   Botvinnik|            10|
|    Steinitz|            10|
|        Euwe|             9|
|      Lasker|             8|
+------------+--------------+
only showing top 15 rows



3. Most and Least Popular eco move in world championship history.
Result attributes: eco, eco_name, number_of_occurences
Final result will have only two rows

In [ ]:
chess_moves.show(5)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
chess_info.show(6)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|  

In [ ]:
eco_codes.show(6)

+---+--------------------+------------------+--------+---------+
|eco|            eco_name|       eco_example|eco_type|eco_group|
+---+--------------------+------------------+--------+---------+
|A00|    Uncommon Opening|1 g4, a3, h3, etc.|       A|    Flank|
|A01|Nimzovich-Larsen ...|              1 b3|       A|    Flank|
|A02|      Bird's Opening|              1 f4|       A|    Flank|
|A03|      Bird's Opening|           1 f4 d5|       A|    Flank|
|A04|        Reti Opening|             1 Nf3|       A|    Flank|
|A05|        Reti Opening|         1 Nf3 Nf6|       A|    Flank|
+---+--------------------+------------------+--------+---------+
only showing top 6 rows



In [ ]:
eco_info=eco_codes.join(chess_info, eco_codes.eco==chess_info.eco,how="inner").drop(eco_codes.eco)

In [ ]:
eco_info_move=eco_info.join(chess_moves, eco_info.game_id==chess_moves.game_id,how="inner").drop(chess_moves.game_id)

In [ ]:
eco_info_move.show(5)

+-----------------+--------------------+--------+--------------------+--------------------+----------+--------------------+---------+-----------+-----+----------------+--------------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+------------+--------------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------

In [ ]:
data=eco_info_move.groupBy("eco","eco_name").count()
data=data.withColumnRenamed("count","number_of_occurences")
data1=data.sort("number_of_occurences",ascending=False)
data1=data1.limit(1)
data2=data.sort("number_of_occurences",ascending=True)
data2=data2.limit(1)

data3=data1.unionByName(data2)
data3.show()

+---+--------------------+--------------------+
|eco|            eco_name|number_of_occurences|
+---+--------------------+--------------------+
|C42|      Petrov Defense|                5599|
|D54|Queen's Gambit De...|                  30|
+---+--------------------+--------------------+



4. Find the eco move with most winnings.
Ps. Use this opening move in your next chess game🙂
Result attributes: eco, eco_name

In [ ]:
df4=eco_info_move.select("eco","eco_name").where("winner != 'draw'")

In [ ]:
df_4= df4.groupBy("eco","eco_name").count()
df_4=df_4.sort("count",ascending=False)
df_4=df_4.select("eco","eco_name")
df_4=df_4.limit(1)
df_4.show()


+---+-----------------+
|eco|         eco_name|
+---+-----------------+
|B90|Sicilian, Najdorf|
+---+-----------------+



5. Longest and shortest game ever played in a world championship in terms of move.
Chess Funda: "move" is completed once both White and Black have played one turn. e.g If a game lasts 10 moves, both White and Black have
played 10 moves)
Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [ ]:
df_pre=chess_moves.groupBy("game_id").count()
df1 = df_pre.sort("count",ascending=False)
df2 = df_pre.sort("count",ascending=True)

In [ ]:
highest_count=df1.limit(1)
lowest_count=df2.limit(1)

In [ ]:
result5= highest_count.unionByName(lowest_count)

In [ ]:
result05=chess_info.join(result5, chess_info1.game_id==result5.game_id, how="right").drop(result5.game_id)

In [ ]:
result05.select("game_id","event","tournament_name","count").show()

+--------------------+--------------------+---------------+-----+
|             game_id|               event|tournament_name|count|
+--------------------+--------------------+---------------+-----+
|4424a0a4-3732-407...|         FIDE WCh KO|  FideChamp2000|  291|
|1846cede-0037-4f0...|World Championshi...| WorldChamp1972|    1|
+--------------------+--------------------+---------------+-----+



6. Shortest and Longest Draw game ever Played.
Result attributes: game_id, event, tournament_name, number_of_moves
Final result will have only two rows

In [ ]:
chess_info1.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|winner1|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+----------+-----+---------+---------------------+---+--------------------+---------------+-------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| Worl

In [ ]:
result6=chess_info1.where("winner1 = 'draw'")

In [ ]:
result06 =result6.groupBy("tournament_name").count()

In [ ]:
df61 = result06.sort("count",ascending=False)
df62 = result06.sort("count",ascending=True)

In [ ]:
highest_count=df61.limit(1)
lowest_count=df62.limit(1)

In [ ]:
result006= highest_count.unionByName(lowest_count)

In [ ]:
result_06=chess_info.join(result006, chess_info1.tournament_name==result006.tournament_name, how="inner").drop(result006.tournament_name)

In [ ]:
a1=result_06.select("game_id","event","tournament_name","count").limit(1)

In [ ]:
highest_count.show()

+---------------+-----+
|tournament_name|count|
+---------------+-----+
|  FideChamp2002|  197|
+---------------+-----+



In [ ]:
result_06.show()

+--------------------+----------+-----------+----------+-----------+-----+-----------------+-------------+-------+---------+---------+-----------+-----------+------------+----------+--------------+---------+---------------------+---+--------------------+---------------+-----+
|             game_id|game_order|      event|      site|date_played|round|            white|        black| result|white_elo|black_elo|white_title|black_title|      winner|winner_elo|         loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|count|
+--------------------+----------+-----------+----------+-----------+-----+-----------------+-------------+-------+---------+---------+-----------+-----------+------------+----------+--------------+---------+---------------------+---+--------------------+---------------+-----+
|cf78855f-23a5-430...|         1|FIDE WCh KO|Moscow RUS| 2001.11.03|  4.1|         Adams,Mi|    Svidler,P|1/2-1/2|     2731|     2686|       null|       null|        dra

7. Most and Least rated Player.
Result attributes: player_name, elo
Chess Funda: elo is the rating of the player in chess tournament.
Final result will have only two rows

In [ ]:
chess_info.show(10)


+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|  

In [ ]:
chess_in1=chess_info.withColumn("black_split",split(col("black"),"[,]"))\
          .withColumn("black1",col("black_split")[0]).drop("black_split","black")
chess_info1=chess_in1.withColumn("white_split",split(col("white"),"[,]"))\
          .withColumn("white1",col("white_split")[0]).drop("white_split","white")

In [ ]:
chess_info1=chess_info1.where("white_elo != 'null'")
chess_info1=chess_info1.where("black_elo != 'null'")

In [ ]:
result7= chess_info1.groupBy("black1").agg(F.avg("black_elo"))
res7 = result7.sort("avg(black_elo)",ascending=False)

result7l=chess_info1.groupBy("white1").agg(F.avg("white_elo"))
res7l = result7l.sort("avg(white_elo)",ascending=True)

In [ ]:

result71=res7.limit(1)
result72=res7l.limit(1) 

df1=result71.withColumnRenamed("black1","name")
df1=df1.withColumnRenamed("avg(black_elo)","avg_elo")

df2=result72.withColumnRenamed("white1","name")
df2=df2.withColumnRenamed("avg(white_elo)","avg_elo")


result7_final= df1.unionByName(df2)

In [ ]:
result7_final.show()

+-------+----------+
|   name|   avg_elo|
+-------+----------+
|Carlsen|2835.40625|
|Abulhul|    2076.0|
+-------+----------+



8. 3rd Last Player with most Loss.
Result attributes: player_name
Final result will have only one row

In [ ]:
chess_info.show(10)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|  

In [ ]:
chess_loser=chess_info.withColumn("loser_split",split(col("loser"),"[,]"))\
          .withColumn("loser1",col("loser_split")[0]).drop("loser_split","loser")
chess_loser=chess_loser.where("loser != 'draw'")

In [ ]:
result8=chess_loser.groupBy("loser1").count()
result8=result8.sort("count",ascending=False)
result8=result8.withColumnRenamed("loser1","player_name")
result8=result8.collect()[2]
result8

Row(player_name='Karpov', count=38)

9. How many times players with low rating won matches with their total win Count.
Result attributes: player_name, win_count

In [ ]:
data= chess_info.where("winner != 'draw'")

In [ ]:
data.show(5)

+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|        site|date_played|round|           white|           black|result|white_elo|black_elo|white_title|black_title|          winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|58e83255-93bb-4d5...|         6|WCh 2021|   Dubai UAE| 2021.12.03|    6|       Carlsen,M|Nepomniachtchi,I|   1-0|     2855|     2782|       null|       null|       Carl

In [ ]:
data=data.select("winner").where("winner_elo < loser_elo")

In [ ]:
data=data.groupBy("winner").count()
result9=data.sort("count",ascending=False)
result9.show()

+--------------------+-----+
|              winner|count|
+--------------------+-----+
|     Karpov, Anatoly|   20|
|      Kasimdzhanov,R|   11|
|           Macieja,B|    9|
|           Kramnik,V|    8|
|  Kortschnoj, Viktor|    8|
|          Grischuk,A|    7|
|        Ponomariov,R|    7|
|Khalifman, Alexander|    6|
|           Ehlvest,J|    5|
|   Akopian, Vladimir|    5|
|      Kasparov, Gary|    5|
|      Short, Nigel D|    5|
|         Dominguez,L|    5|
|           Sulskis,S|    5|
|           Gelfand,B|    4|
|          Nakamura,H|    4|
| Tkachiev, Vladislav|    4|
|            Volkov,S|    4|
|Nisipeanu, Liviu ...|    4|
|         Filippov,Va|    4|
+--------------------+-----+
only showing top 20 rows



10. Move Sequence for Each Player in a Match.
Result attributes: game_id, player_name, move_sequence, move_count

In [ ]:
chess_moves.show(10)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
from pyspark.sql.functions import split, reverse
chess_move1=chess_moves.withColumn("move_sequence_split",split(col("move_sequence"),"[|]"))\
          .withColumn("last_move_sequence",reverse(col("move_sequence_split"))[0]).drop("move_sequence_split","move_sequence")

In [ ]:
chess_move1.show(6)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
result10=chess_move1.select("game_id","player","last_move_sequence","move_no")

In [ ]:
result10.show(5)

+--------------------+----------------+------------------+-------+
|             game_id|          player|last_move_sequence|move_no|
+--------------------+----------------+------------------+-------+
|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|                e4|      1|
|86e0b7f5-7b94-4ae...|       Carlsen,M|                e5|      2|
|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|               Nf3|      3|
|86e0b7f5-7b94-4ae...|       Carlsen,M|               Nc6|      4|
|86e0b7f5-7b94-4ae...|Nepomniachtchi,I|               Bb5|      5|
+--------------------+----------------+------------------+-------+
only showing top 5 rows



11. Total Number of games where losing player has more Captured score than Winning player.
Hint: Captured score is cumulative, i.e., for 3rd capture it will have score for 1, 2, and 3rd.
Result attributes: total_number_of_games Final result will have only one row

In [ ]:
chess_info.show(6)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|  

In [ ]:
#converting white and black column to only first name
df_info=chess_info.withColumn("black_split",split(col("black"),"[,]"))\
          .withColumn("black1",col("black_split")[0]).drop("black_split","black")
df_info=df_info.withColumn("white_split",split(col("white"),"[,]"))\
          .withColumn("white1",col("white_split")[0]).drop("white_split","white")

In [ ]:
df_info.show(5)


+--------------------+----------+--------+---------+-----------+-----+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+--------------+--------------+
|             game_id|game_order|   event|     site|date_played|round| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|        black1|        white1|
+--------------------+----------+--------+---------+-----------+-----+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+--------------+--------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| WorldChamp2021|       Carlsen|Nepomniachtchi|


In [ ]:
import sys 
from pyspark.sql.window import Window 
import pyspark.sql.functions as F 
df_info111=df_info.where("winner != 'draw'")

cum_sum_df = df_info111.withColumn('winner_elo_cum_sum',F.sum(df_info111.winner_elo).over(Window.partitionBy("winner").orderBy().rowsBetween(-sys.maxsize,0)))
cum_sum_df =cum_sum_df.withColumn('loser_elo_cum_sum',F.sum(cum_sum_df.loser_elo).over(Window.partitionBy("loser").orderBy().rowsBetween(-sys.maxsize,0)))
cum_sum_df.show(5)

+--------------------+----------+-------------+---------+-----------+-----+------+---------+---------+-----------+-----------+--------------------+----------+-----------------+---------+---------------------+---+--------------------+---------------+---------+---------+------------------+-----------------+
|             game_id|game_order|        event|     site|date_played|round|result|white_elo|black_elo|white_title|black_title|              winner|winner_elo|            loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|   black1|   white1|winner_elo_cum_sum|loser_elo_cum_sum|
+--------------------+----------+-------------+---------+-----------+-----+------+---------+---------+-----------+-----------+--------------------+----------+-----------------+---------+---------------------+---+--------------------+---------------+---------+---------+------------------+-----------------+
|5524a0a5-b159-4fc...|       122|FIDE-Wch k.o.|Las Vegas| 1999.08.03|  2.1|   0

In [ ]:
df_11=cum_sum_df.where("loser_elo_cum_sum > winner_elo_cum_sum")
result_11=df_11.count()
result_11

321

12. List All Perfect Tournament with Winner Name.
Chess Funda: Perfect Tournament means a player has won all the matches excluding draw matches. e.g Player A has won 5 matches out of 7
Matches in tournament where 2 matches are draw and player B has won 0 matches)
Result attributes: winner_name, tournament_name

In [ ]:
chess_info.show(5)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| 

In [ ]:
prft=chess_info.withColumn("result_split",split(col("result"),"[-]"))\
          .withColumn("resultw",col("result_split")[0]).drop("result_split")

prft=prft.withColumn("result_split",split(col("result"),"[-]"))\
          .withColumn("resultb",col("result_split")[1]).drop("result_split","result")

In [ ]:
prft=prft.where("winner != 'draw'")

In [ ]:
prft.show(5)

+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+-------+-------+
|             game_id|game_order|   event|        site|date_played|round|           white|           black|white_elo|black_elo|white_title|black_title|          winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|resultw|resultb|
+--------------------+----------+--------+------------+-----------+-----+----------------+----------------+---------+---------+-----------+-----------+----------------+----------+----------------+---------+---------------------+---+--------------------+---------------+-------+-------+
|58e83255-93bb-4d5...|         6|WCh 2021|   Dubai UAE| 2021.12.03|    6|       Carlsen,M|Nepomniachtchi,I|     2855|     2782|       null|   

In [ ]:
a=prft.groupBy("tournament_name","winner").agg(F.sum("resultw"),F.sum("resultb"))

In [ ]:
a1=a.where("sum(resultb) = 0")
a1=a1.select("tournament_name","winner")
a2=a.where("sum(resultw) =0")
a2=a2.select("tournament_name","winner")

In [ ]:
result12=a1.unionByName(a2)

In [ ]:
result12.show()

+---------------+------------------+
|tournament_name|            winner|
+---------------+------------------+
|  FideChamp2002|          Nataf,IA|
|  FideChamp1999|Ponomariov, Ruslan|
|  FideChamp2002|    Vallejo Pons,F|
|  FideChamp2004|        Malakhov,V|
|  FideChamp2002|      Radziewicz,I|
|  FideChamp2002|     Baginskaite,C|
|  FideChamp2002|      Prudnikova,S|
|  FideChamp1999|   Wahls, Matthias|
|  FideChamp1998|     Ehlvest, Jaan|
| WorldChamp2008|         Kramnik,V|
|  FideChamp1999|     Ehlvest, Jaan|
|  FideChamp1998|      Glek, Igor V|
|  FideChamp2004|        Inarkiev,E|
|  FideChamp2000|      Villamayor,B|
|  FideChamp2002|        Gurevich,M|
|   PCAChamp1995|Anand, Viswanathan|
|  FideChamp2002|       Savchenko,S|
|  FideChamp2000|         Chernin,A|
|  FideChamp1999| Babula, Vlastimil|
|  FideChamp1999|    Benjamin, Joel|
+---------------+------------------+
only showing top 20 rows



13. Player with highest winning ratio.
Hint: Winning ratio: (Number of rounds won)/(Number of rounds played)
Result attributes: player_name
Final result will have only one row

In [ ]:
chess_info.show(10)

+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|             game_id|game_order|   event|     site|date_played|round|           white|           black| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|
+--------------------+----------+--------+---------+-----------+-----+----------------+----------------+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|Nepomniachtchi,I|       Carlsen,M|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|  

In [ ]:
chess_info12=chess_info.withColumn("white_split",split(col("white"),"[,]"))\
          .withColumn("whitew",col("white_split")[0]).drop("white_split","white")

chess_info12=chess_info12.withColumn("black_split",split(col("black"),"[,]"))\
          .withColumn("blackw",col("black_split")[0]).drop("black_split","black")

In [ ]:
chess_info12.show()

+--------------------+----------+--------+----------+-----------+-----+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+--------------+--------------+
|             game_id|game_order|   event|      site|date_played|round| result|white_elo|black_elo|white_title|black_title|   winner|winner_elo|           loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|        whitew|        blackw|
+--------------------+----------+--------+----------+-----------+-----+-------+---------+---------+-----------+-----------+---------+----------+----------------+---------+---------------------+---+--------------------+---------------+--------------+--------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021| Dubai UAE| 2021.11.26|    1|1/2-1/2|     2782|     2855|       null|       null|     draw|      null|            draw|     null|                    0|C88|2022-07-

In [ ]:
a1=chess_info12.groupBy("whitew").count()
a1=a1.withColumnRenamed("whitew","name")
a1=a1.withColumnRenamed("count","countw")


a2=chess_info12.groupBy("blackw").count()
a2=a2.withColumnRenamed("blackw","name")
a2=a2.withColumnRenamed("count","countb")

In [ ]:
a=a1.join(a2,a1.name==a2.name, how ="inner").drop(a2.name)

In [ ]:
a.show(5)

+--------------+------+------+
|          name|countw|countb|
+--------------+------+------+
|Becerra Rivero|     1|     1|
|     Giorgadze|     4|     5|
|         Milov|    13|    14|
|        Elarbi|     1|     1|
|        Babula|     2|     2|
+--------------+------+------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import col

df_13=a.select("name",((col("countw") + col("countb"))).alias("rounds_played"))
df_13.show(30)

+--------------+-------------+
|          name|rounds_played|
+--------------+-------------+
|Becerra Rivero|            2|
|     Giorgadze|            9|
|         Milov|           27|
|        Elarbi|            2|
|        Babula|            4|
|         Kritz|            4|
|       Fominyh|            2|
|        Asabri|            2|
|    Morozevich|           48|
|        Kobese|            7|
|  Peng Xiaomin|           16|
|      Karjakin|           18|
|     Neelotpal|            4|
|       Psakhis|           25|
|         Safin|            2|
|          Lima|            6|
|        Lerner|            2|
|   Gu Xiaobing|            2|
|         Gulko|           27|
|     Arakhamia|            2|
|    Hjartarson|            8|
|       Mahjoob|            4|
|        Volkov|           16|
|      Tkachiev|           48|
|   Yu Shaoteng|            4|
|         Salov|            6|
|   Miladinovic|            6|
|       Lautier|           42|
|    Timoshenko|            2|
|       

In [ ]:
chess_info12.show(5)

+--------------------+----------+--------+---------+-----------+-----+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+--------------+--------------+
|             game_id|game_order|   event|     site|date_played|round| result|white_elo|black_elo|white_title|black_title|winner|winner_elo|loser|loser_elo|winner_loser_elo_diff|eco|        date_created|tournament_name|        whitew|        blackw|
+--------------------+----------+--------+---------+-----------+-----+-------+---------+---------+-----------+-----------+------+----------+-----+---------+---------------------+---+--------------------+---------------+--------------+--------------+
|86e0b7f5-7b94-4ae...|         1|WCh 2021|Dubai UAE| 2021.11.26|    1|1/2-1/2|     2782|     2855|       null|       null|  draw|      null| draw|     null|                    0|C88|2022-07-22T22:33:...| WorldChamp2021|Nepomniachtchi|       Carlsen|


In [ ]:
chess_info12=chess_info12.where("winner !='draw'")
chess_info12=chess_info12.withColumn("winner_split",split(col("winner"),"[,]"))\
          .withColumn("winnerw",col("winner_split")[0]).drop("winner_split","winner")

In [ ]:
b1=chess_info12.select("winnerw").where("whitew = winnerw")
b1=b1.groupBy("winnerw").count()
b1=b1.withColumnRenamed("count","countw")

b2=chess_info12.select("winnerw").where("blackw = winnerw")
b2=b2.groupBy("winnerw").count()
b2=b2.withColumnRenamed("count","countb")

In [ ]:
b=b2.join(b1,b1.winnerw==b2.winnerw, how ="inner").drop(b2.winnerw)

In [ ]:
b.show()

+------+-------------+------+
|countb|      winnerw|countw|
+------+-------------+------+
|     3|        Milov|     6|
|     1|        Kritz|     1|
|     4|   Morozevich|    11|
|     2| Peng Xiaomin|     3|
|     2|      Psakhis|     2|
|     2|        Gulko|     2|
|     5|     Tkachiev|     9|
|     2|       Volkov|     2|
|     1|  Miladinovic|     1|
|     7|      Lautier|     5|
|     1|      Nijboer|     1|
|     1|Granda Zuniga|     1|
|     1|       Serper|     2|
|     7|     Kasparov|    24|
|     1|       Jobava|     1|
|     2|     Benjamin|     2|
|     1|      Antonio|     1|
|     2|    Andersson|     3|
|     1|     Tarrasch|     2|
|     4|Ye Jiangchuan|     4|
+------+-------------+------+
only showing top 20 rows



In [ ]:
df_13a=b.select("winnerw",((col("countw") + col("countb"))).alias("rounds_won"))
df_13a.show(30)

+-------------+----------+
|      winnerw|rounds_won|
+-------------+----------+
|        Milov|         9|
|        Kritz|         2|
|   Morozevich|        15|
| Peng Xiaomin|         5|
|      Psakhis|         4|
|        Gulko|         4|
|     Tkachiev|        14|
|       Volkov|         4|
|  Miladinovic|         2|
|      Lautier|        12|
|      Nijboer|         2|
|Granda Zuniga|         2|
|       Serper|         3|
|     Kasparov|        31|
|       Jobava|         2|
|     Benjamin|         4|
|      Antonio|         2|
|    Andersson|         5|
|     Tarrasch|         3|
|Ye Jiangchuan|         8|
|       Sadler|         2|
| Kasimdzhanov|        19|
|   Kortschnoj|        13|
|          Acs|         3|
|      Smirnov|         5|
|      Sulskis|         5|
|    Sasikiran|         2|
|   Mamedyarov|         3|
|   Ponomariov|        14|
|     Steinitz|        43|
+-------------+----------+
only showing top 30 rows



In [ ]:
result13=df_13.join(df_13a,df_13.name==df_13a.winnerw, how="inner").drop(df_13a.winnerw)

In [ ]:
result13=result13.select("name",((col("rounds_won")/ col("rounds_played"))).alias("winning_ratio"))
result13=result13.sort("winning_ratio",ascending=False)
result13=result13.limit(1)
result13.show()

+------+-------------+
|  name|winning_ratio|
+------+-------------+
|Foisor|          1.0|
+------+-------------+



14. Player who had given checkmate with Pawn.
Note: Consider all events for this query
Result attributes: player_name
Final result will have only one row

In [ ]:
chess_moves.show()

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
result_14 = chess_moves.where("is_check_mate != 0")
result_14 = result_14.where("piece = 'P'")

In [ ]:
result_14=chess_info.join(result_14, chess_info.game_id==result_14.game_id, how="inner").drop(result_14.game_id)

In [ ]:
result_14=result_14.select("winner")
result_14.show()

+--------------+
|        winner|
+--------------+
|Andersson, Ulf|
+--------------+



15. List games where player has won game without queen.
Result attributes: game_id, event, player_name

In [ ]:
chess_moves.show(10)

+--------------------+-------+------------+----------------+--------+----+-----------+---------+-----+-----+--------------------+--------+-------------+--------------+----------------------+------------+------------------------+-----------+-----------+----------------+----------------+-----------------+-----------------+------------------+------------------+------------------+------------------+----------------+----------------+------------------------+------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------

In [ ]:
df=chess_moves.where("white_queen_count = '0'")
df=df.where("black_queen_count = '0'")

In [ ]:
df=df.select("game_id","player")

In [ ]:
df=chess_info.join(df, chess_info.game_id==df.game_id, how="inner").drop(df.game_id)

In [ ]:
df=df.where("winner != 'draw'")

In [ ]:
df=df.distinct()

In [ ]:
result_15=df.select("game_id","event","winner")

In [ ]:
result_15.show(10,truncate=False)

+------------------------------------+-----------------+-----------------+
|game_id                             |event            |winner           |
+------------------------------------+-----------------+-----------------+
|f0ad2ebb-a299-4b37-bb8f-5c332bbbce7d|FIDE WCh KO      |Smirnov,P        |
|4a2618a6-af36-48ea-8676-73a34b1c7d96|FIDE WCh KO      |Xu Jun           |
|14d0dcf4-356e-4ec8-a8ba-e430f1dcf699|FIDE WCh KO      |Vladimirov,E     |
|2c36f39f-4401-4985-a8cc-1e2a7072cb45|FIDE-Wch k.o.    |Glek, Igor V     |
|e7d10f1f-5120-444f-969c-26f4166fcf31|WCh-FIDE         |Anand,V          |
|7e04a9d6-d522-46bb-9696-7c19c2d487ed|FIDE WCh KO      |Azmaiparashvili,Z|
|291aba1f-80ad-45da-951c-b5882a82b148|FIDE WCh KO      |Zhang Pengxiang  |
|c360aced-29c6-4964-b160-7dc617b953e5|FIDE WCh KO Final|Ponomariov,R     |
|304f830c-0d1e-4a48-a60e-f7971487c2f5|FIDE WCh KO      |Serper,G         |
|ce429291-142b-4fb9-8a68-c1e9b78be6a0|FIDE-Wch         |Karpov, Anatoly  |
+------------------------

**SECTION 3**

In [ ]:
!pip install pydrive

In [ ]:
def create_file(listt): 
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth

  import os 
  gauth = GoogleAuth()
  
  gauth.LocalWebserverAuth()       
  drive = GoogleDrive(gauth)

  path = r"gdrive/MyDrive/DE_SOLUTION_Gopika_Jayaram"  
  for x in os.listdir(path):
   
    f = drive.CreateFile({'title': "DE_SOLUTION_Gopika_Jayaram"})
    f.SetContentFile(os.path.join(path, "DE_SOLUTION_Gopika_Jayaram"))
    f.Upload()
  
  pandas_df=[]
  no=1
  for i in listt:
      
      pandas_df.append(i.toPandas())

      from google.colab import drive

      drive.mount('/content/drive')
      path = 'gdrive/MyDrive/DE_SOLUTION_Gopika_Jayaram'
      
      
      f="DE_SOLUTION_Gopika_Jayaram/df."+f'{no}'+".csv"
      %notebook+=1
      with open(path, 'w', encoding = 'utf-8-sig') as f:
        i.to_csv(f)

      

